# Exercise Sheet 3: Head Modeling & Non-linear dynamics

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import signal as signal

In [2]:
# Define useful functions and import head models and source distributions
def logistic(h,a=1):
    return 1/(1+np.exp(-a*h))


def phi_dip(r, Q, r_0=None,sigma=0.33):
    r=np.array(r)
    Q=np.array(Q)    
    if r_0 is not None:
        r_0=np.array(r_0)
        r=(r-r_0).T
    return 1/(4*np.pi*sigma)*(np.tensordot(Q,r,1))/np.power(np.linalg.norm(r,axis=0),3)


pos1=np.load('bnd4_pos1.npy')
tri1=np.load('bnd4_tri1.npy')
cortex2dsm=np.load('cortex2dsm.npy')
gridpos=np.load('gridpos.npy')
NoTri=tri1.shape[0]
NoPnt=pos1.shape[0]

## Task 1: BEM vs infinite solution  (5 points)


The file 'cortex2dsm.npy', lets call it again $L$ is similar to the leadfields of the last sheet but this time mapping to the cortex surface. Again, you have to select a dipole and the dipolar moments $q$. The voltage is then calculated by $v=L_i \cdot q$.

**Tasks:**

a) Chose an arbitrary dipole and a dipolar moment yourself and simulate the field using the BEM head model.

b) Then, calculate the anayltical solution for an infinite homogeneous current dipole at the cortex surface vertices (positions) for the same source location and dipolar moment. Source conductivity is $\sigma_1=0.201$. You can find the source position $r_q$ for the dipole in 'gridpos.npy' which has the same indexing as the leadfield.

c) Plot the distribution of the potential on the brain surface using the *plot_trisurf* function similar as in task 7 sheet 2 in combination with indiviudally colored triangles.

c) Also, use the simple *plot* function to plot the voltage for both the analytic solution in infiinite space and the BEM soution on the cortex surface over the vertex index to get a closer look at individual values.

Are the solutions equal? Investigate and explain the difference. 

In [3]:
S_idx = 2500
Q1 = np.array([10,10,10])*1e-15
r_q = gridpos[S_idx, :]
sigma1 = 0.201


phis = cortex2dsm[:, S_idx, :] @ Q1
surface_phis = np.zeros(tri1.shape[0])


phis2 =  phi_dip(pos1, Q1, r_0=r_q, sigma=sigma1)
surface_phis2 = np.zeros(tri1.shape[0])


# Average over the potentials at the points to get the potentials for the triangles.
for i, tri in enumerate(tri1):
    surface_phis[i] = np.mean(cortex2dsm[tri, S_idx, :], axis=0) @ Q1
    surface_phis2[i] = np.mean(phis2[tri])
    

# Build a colormap.
absmax=np.max(np.abs([phis]))
mappy=plt.cm.ScalarMappable(cmap=plt.cm.jet)
mappy.set_clim([-absmax,absmax])
absmax2=np.max(np.abs([phis2]))
mappy2=plt.cm.ScalarMappable(cmap=plt.cm.jet)
mappy2.set_clim([-absmax,absmax])
# mappy.set_array([]) #not sure what this does


# Plot surf and set the surface colors.
fig = plt.figure(figsize=[15,15])
ax = fig.add_subplot(221, projection='3d')
ax.set_title("BEM Solution on Cortex Surface", size=17)
trisufplt=ax.plot_trisurf(
    pos1[:, 0], 
    pos1[:, 1], 
    pos1[:, 2], 
    triangles=tri1,
    alpha=0.4,
    edgecolor='k',
    linewidth=0.4
)
trisufplt.set_facecolors(mappy.to_rgba(surface_phis))
plt.colorbar(mappy, ax=ax, fraction=0.03, pad=0.1)


ax2 = fig.add_subplot(222, projection='3d')
ax2.set_title("Analytical Solution on Cortex Surface", size=17)
trisufplt2=ax2.plot_trisurf(
    pos1[:, 0], 
    pos1[:, 1], 
    pos1[:, 2], 
    triangles=tri1,
    alpha=0.4,
    edgecolor='k',
    linewidth=0.4
)
trisufplt2.set_facecolors(mappy2.to_rgba(surface_phis2))
plt.colorbar(mappy2, ax=ax2, fraction=0.03, pad=0.1)


ax3 = fig.add_subplot(223)
ax3.set_title("Voltage Distribution of BEM Solution", size=17, pad=15)
ax3.plot(phis, color='mediumpurple')
ax3.set_ylabel("potential [V]", size=15)
ax3.set_xlabel("Vertex Index", size=15)
ax3.set_ylim(np.min((phis, phis2))-1e-12, np.max((phis, phis2))+1e-12)
ax3.grid()
ax4 = fig.add_subplot(224)
ax4.set_title("Voltage Distribution of Analytical Solution", size=17, pad=15)
ax4.plot(phis2, color='mediumpurple')
ax4.set_ylabel("potential [V]", size=15)
ax4.set_xlabel("Vertex Index", size=15)
ax4.set_ylim(np.min((phis, phis2))-1e-12, np.max((phis, phis2))+1e-12)
ax4.grid()
plt.show()

ValueError: Unknown projection '3d'

<Figure size 1080x1080 with 0 Axes>

Here we see the solutions are not equal. The analytical solution seems to exhibit larger amounts of positive potential distributed along the vertices, as well as less negative potential. It is worth noting that the maxima and minima are more extreme in the analytical solution. The analytical solution is also more granular, whereas the BEM solution is more smooth. This would coincide with the idea that our BEM solution has taken into account for the effects of volume conduction between the cortex and CSF. Since the CSF is much more conductive, we know that the flow of current will shift towards the boundary (towards the CSF). This could account for the less extreme values in the voltage distributions as well as the wider distribution of values accros the cortex. 

## Task 2: The effect of the non-linear transfer function: simulation (5 points)


**Tasks:**

a) Write a function simulating a sine-wave $x(t)=a*sin(\omega t)+b=a*sin(2\pi f t)+b$. Simulate a time interval of 10s with a sampling rate of fs=200Hz and plot it over the first $0.5s$ with $a=1$ for the amplitude, $b=0$ for the offset and $f=10Hz$ for the frequency.

b) Calculate the Power-Spectral-Density (PSD) of the signal and plot it over the frequency. For calculating the PSD, use the welch algorithm (scipy.signal.welch) and plot it using a logarithmic scale (10*log10(Pxx)). Clip the PSD at -80dB. Use a 10s snippet of the signal for calculating the PSD.

c) Use the logistic function $y(x)=\frac{1}{1+e^{-ax}}$ on the signal $x(t)$ as a non-linear time-invariant amplitude transformation. Do this for all possible combinations of the following amplitudes and offsets of the sine wave: a={0.5, 1, 10} and b={-2,-1,0, 1, 2}. Calculate and plot the PSD as in b).

d) The signal in a) and b) is a pure sine wave, which as a fourier transform has a single peak in the frequency spectrum. Explain the differences to the PSD of the sine wave (b) and the effect of the non-linear transformation on the PSD (c).

In [ ]:
def sine_wave(t, amp, freq, offset):
    return amp * np.sin(2 * np.pi * freq * t) + b

samp_freq = 200
t_range = np.arange(0, 10, 1/samp_freq)
a = 1
b = 0
f = 10
ex = sine_wave(t_range, a, f, b)
freqs, Pxx = signal.welch(ex, samp_freq)

fig = plt.figure(figsize=(18, 3))
ax = fig.add_subplot(131)
ax.set_title("Example Sine Wave", size=15)
ax.plot(t_range, ex, c='mediumpurple')
ax.set_xlim(0, 0.5)
ax.set_xlabel('time [s]')
ax.set_ylabel('amplitube [a.u.]')
ax.grid()
ax2 = fig.add_subplot(132)
ax2.set_title("Example PSD of Sine Wave", size=15)
ax2.plot(freqs, 10*np.log10(Pxx), c='forestgreen')
ax2.set_ylim(-80, 10)
ax2.set_xlabel("frequency [Hz]")
ax2.set_ylabel("power spectrum [dB]")
ax2.grid()
ax3 = fig.add_subplot(133)
ax3.set_title("Example of Transformed Sine Wave", size=15)
ax3.plot(t_range, logistic(ex), c='mediumpurple')
ax3.set_xlim(0, 0.5)
ax3.set_xlabel('time [s]')
ax3.set_ylabel('amplitube [a.u.]')
ax3.grid()
plt.show()

NOTE: Here we can see that the logistic transformation of our signal results in a signal between 0 and 1. This will be the case for all combinations of offsets and amplitudes as the logistic function has this property of transforming the signal relative to its limits. Since we know that the offsets will shift the signal up or down and that amplitudes will stretch or squish them, there is no need to plot what they will look like. In additions, these signal will be transformed to be between 0 and 1, so we know the range of our resulting signal. Although, if the amplitude of the signal is much greater than 1, then the logistic function will transform the signal into a step like oscillating signal. Instead, it is more interesting to investigate how the amplitude and offset changes affect the PSD of the transformed signal.

In [ ]:
a_range = [0.5, 1, 10]
b_range = [-2, -1, 0, 1, 2]
colors = ['indigo', 'mediumpurple', 'purple', 'mediumpurple', 'indigo']
samp_freq = 200
t_range = np.arange(0, 10, 1/samp_freq)
f = 10


fig = plt.figure(figsize=(18, 12))
for i, a in enumerate(a_range):
    for j, b in enumerate(b_range):
        orig_signal = sine_wave(t_range, a, f, b)
        transformed_signal = logistic(orig_signal)
        freqs, Pxx = signal.welch(transformed_signal, samp_freq)
        ax = fig.add_subplot(len(a_range), len(b_range), i*len(b_range) + j + 1)
        ax.set_title(f'a = {a}, b = {b}')
        ax.plot(freqs, 10*np.log10(Pxx), c=colors[j])
        ax.set_ylim(-80, 0)
        # Comment above and use below to visualize transformations
#         ax.plot(t_range, transformed_signal)
#         ax.set_xlim(0, 0.5)
        ax.set_xlabel("frequency [Hz]") if i == (len(a_range) - 1) else None
        ax.set_ylabel("power spectrum [dB]") if j == 0 else None
        ax.grid()
        

There are a few differences between the PSD of the pure sine wave and the PSD of the non-linearly transformed sine wave. The first thing to note is that the PSD of the pure sine wave, or any linear transformation of one, has a single peak. The moment we introduce a non-linearity to the wave, we start to see harmonic frequencies appear in the PSD. As we increase the amplitude of our sine wave, the effects of the non-linear transformation become more dramatic, and we need more harmonics to describe the shape of the signal. If we introduce an offset, the effects of the logisitic tranformation increase as well and we start to see even more harmonics pop up in the PSD. It is worth noting that the symmetric offsets produce identical PSDs. This is a result of the same shape of the transformed signal at opposite offsets.

## Task 3: The effect of the non-linear transfer function: analytic & simulation (5 points)

With a little trick, the clipping of an oscillating signal like a cosine wave y(t) can be seen as a linear operation: the multiplication with a rectangular oscillation x(t) of same frequency and apropriate phase. We ignore the DC part in this exercise. This models the "on/off" effect of the clipping:

$z(t)=x(t)y(t)$

For the periodic rectangular time series defined within a period $T_0=\frac{1}{f_0}$ (note that this is a little bit different than in the lecture):

$x (t) = \left\{
\begin{array}{ll}
1  & \, \textrm{for} \left|t\right| \leq T_1 \\
0 & \, \textrm{else} \\
\end{array}
\right.$

the corresponding fourier transform (here one-sided) is based on the delta function:

$\hat{x}(f)=\frac{T_1}{T_0} \delta(f)+\sum_{n=1}^\infty \frac{\sin^2(\pi n\frac{T_1}{T_0})}{\pi n} \delta(f-\frac{n}{T_0})$

A cosine wave of frequency $f_0$ has the fourier transform:

$\hat{y}(f)=2 \pi \delta (f-f_0)$

In the theory of Fourier Transformation, a multiplicatiion in time domain corresponds to a convolution in the frequency:

$z(t)=x(t)y(t) <=> z(f)=\hat{x}(f) * \hat{y}(f)$

with the convolution being:

$\hat{x}(f)*\hat{y}(f)=\int_{-\infty}^{\infty}\hat{x}(f-f')\hat{y}(f')df'$

**Task:**

a) Calculate the Fourier Transform $\hat{z}(f)$ of the signal $z(t)$ analyitcally by convolving $\hat{x}(f)$ and $\hat{y}(f)$ in the frequency domain.

b) Implement the corresponding function for plotting and plot the amplitde spectrum for a clipped 10Hz. Set $T_1$ consequently to $\frac{1}{4}T_0$, $\frac{1}{2}T_0$ and $\frac{3}{4}T_0$. Plot the delta function using the matplot functiion stem. What are the corresponding clipping values?

c) Investigate the values at $nf_0$ and their dependency on the frequency.
What is the effect of the clipping in the spectrum? What implications does the non-linear effect of clipping have on clipped sine waves (single frequency peaks) and white noise (spectrum is constant over frequency)?


*Hint: For the convolution, think about the special properties of the dirac delta function $\delta(x)$.

Solution to a):

$\hat{z}(f)=\hat{x}(f)*\hat{y}(f)=\int_{-\infty}^{\infty}\hat{x}(f-f')\hat{y}(f')df'=\int_{-\infty}^{\infty}\hat{x}(f-f')2 \pi \delta (f'-f_0)df'=2\pi\int_{-\infty}^{\infty}\hat{x}(f-f')\delta (f'-f_0)df'=2\pi\hat{x}(f-f_0)=2\pi\left(\frac{T_1}{T_0} \delta(f-f_0)+\sum_{n=1}^\infty \frac{\sin^2(\pi n\frac{T_1}{T_0})}{\pi n} \delta(f-f_0-\frac{n}{T_0})\right)$

In [ ]:
samp_freq = 200
f_0 = 10
t_0 = 1 / f_0
t1_range = [1/4, 1/2, 3/4]
t_range = np.arange(0, 10, 1/samp_freq)
f_max = 100
n = f_max // f_0 - 1


def clipped_wave(f_max, f_0, t1, n):
    t_0 = 1/f_0
    t1 *= t_0
    sigma = 0
    for i in range(1, n):
        sigma += np.sin(np.pi*i*t1/t_0)**2/(np.pi*i)*signal.unit_impulse(f_max, int(f_0 + i/t_0))
    return 2*np.pi*((t1/t_0)*signal.unit_impulse(f_max, f_0) + sigma)

In [ ]:
fig = plt.figure(figsize=(15, 12))
fig.subplots_adjust(hspace=0.5)
for i, t1 in enumerate(t1_range):
    sig = np.cos(2*np.pi*f_0*t_range)*(signal.square(2*np.pi*f_0*t_range, t1) + 1)/2
    ax = fig.add_subplot(3,2, 1 + 2*i)
    ax.set_title(r'$T_1$ = ' + f'{t1}' + r'$T_0$', size=15)
    ax.plot(t_range, sig, c='mediumpurple')
    ax.set_ylim(-1.1, 1.1)
    ax.set_xlim(0, 0.5)
    ax.set_xlabel('time [s]')
    ax.set_ylabel('amplitude [a.u.]')
    ax.grid()
    ax = fig.add_subplot(3,2, 2 + 2*i)
    ax.set_title(r'$T_1$ = ' + f'{t1}' + r'$T_0$', size=15)
    ax.stem(clipped_wave(f_max, f_0, t1, n))
    ax.set_xlabel('freqency [Hz]')
    ax.set_ylabel('amplitude spectrum [a.u.]')
    ax.grid()
plt.show()

The corresponding clipping values for $T_1$ equals $\frac{1}{4}T_0$, $\frac{1}{2}T_0$ and $\frac{3}{4}T_0$ are $0$, $-1$, and $0$ respectively. As we can see from the amplitude spectrum, we have harmonic frequencies appearing. This implies that our signal is no longer a pure sinusoidal signal, thus the effects of clipping using a step function is a non-linear transformation. As we change the length of the on/off portions of our step function, i.e when the signal is clipped, we can observe changes in the harmonic frequencies that emerge. At $T_1 = \frac{1}{4}T_0$ and $T_1 = \frac{3}{4}T_0$, we can see a mixture of the even and odd harmonics that characterize our clipped signal; they seem to have the nearly identical harmonics with the only difference being the in the power of the fundamental frequency. Interestingly enough, when $T_1 = \frac{1}{2}T_0$, apart from the fundamental frequency, only the even number harmonics appear.

The implications of non-linear clipping of the sine signal are that the power of the single peaks decrease and new peaks start to emerge. The implications of non-linear clipping on white noise are that the power spectrum will change from something constant to a spectrum with emerging peaks and that the values at the clipping value are now more likely and so the power at those freqencies will increase.